In [1]:
#Basic Import
import pandas as pd
import numpy as np
import requests
import json
#import keyboard
import os
import threading
import signal
import sys
import time
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import re
#Import for Selenium
import logging

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
search_url = 'https://danbolig.dk/soeg/?o=newest-desc'

#To Break while loops
is_active = True
list_of_links = []

# Create Selenium functions

**save_webriver_content:** save the 

**break_on_escape:** when escape is pressed, the scroller stops.

**selenium_scroller:** open Chrome winow, wait 15 seconds, scroll to bottom. Return links



In [2]:
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    filename='script_log.txt',
                    filemode='w')
def save_links(content):
    with open('list_of_links.txt', 'w') as file:
        file.write('\n'.join(content) + ',' + '\n')
        
def pause_program():
    global is_active
    is_active = False

def on_escape_press(event):
    if event.name == 'esc':
        pause_program()

keyboard.on_press(on_escape_press)

def get_links(driver):
    links = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/bolig/')]")
    list_of_links = [link.get_attribute('href') for link in links]
    return list_of_links

def selenium_scroller(search_url):
    global is_active
    is_active = True
    driver = webdriver.Chrome()
    url = search_url
    driver.get(url)
    scroll_height = 0
    last_height = driver.execute_script("return document.body.scrollHeight")
    time.sleep(15)
    property_listings_div = driver.find_element(By.CSS_SELECTOR, 'div.t-search__list')
    #list_of_links = []
    while is_active:
        try:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", property_listings_div)
            time.sleep(1)
            new_scroll_height = driver.execute_script("return arguments[0].scrollHeight;", property_listings_div)
            if new_scroll_height == scroll_height:
                break
            scroll_height = new_scroll_height
        except KeyboardInterrupt:
            pause_program()
            break  
        except WebDriverException as e:
            logging.error(f"Selenium error: {e}")
            driver.save_screenshot("screenshot.png")
        except Exception as e:
            logging.exception("An unexpected error occurred")
            driver.save_screenshot("error_screenshot.png")
        except WebDriverException as e:
            logging.error(f"Selenium error: {e}")
            driver.save_screenshot("screenshot.png")
            driver.quit()  # Close the driver
        except Exception as e:
            logging.exception("An unexpected error occurred")
            driver.save_screenshot("error_screenshot.png")
            driver.quit()  # Close the driver
    list_of_links = get_links(driver)
    # Collect links and content
    
    
#    driver.quit()  # Close the driver
    
    return list_of_links
 


## Get, save and print list of links
Press escape to break scrolling.

In [ ]:
content = selenium_scroller(search_url)
save_links(content)

https://danbolig.dk/bolig/vejle/7100/villa/0950001202-095/
https://danbolig.dk/bolig/kolding/6000/lejlighed/0170000991-017/
https://danbolig.dk/bolig/aarhus/8200/lejlighed/2730000448-273/
https://danbolig.dk/bolig/kerteminde/5290/villa/2590001307-259/
https://danbolig.dk/bolig/vordingborg/4760/villa/2040000675-204/
https://danbolig.dk/bolig/mariagerfjord/9500/villa/4750000371-475/
https://danbolig.dk/bolig/halsnaes/3390/villa/2930000213-293/
https://danbolig.dk/bolig/vesthimmerlands/9600/villa/2580000393-258/
https://danbolig.dk/bolig/odense/5260/villa/2590001283-259/
https://danbolig.dk/bolig/ishoej/2635/villa/2760000207-276/
https://danbolig.dk/bolig/varde/6862/fritidsbolig/1140000327-114/
https://danbolig.dk/bolig/slagelse/4200/fritidsbolig/2840000399-284/
https://danbolig.dk/bolig/aarhus/8210/villa/2820000519-282/
https://danbolig.dk/bolig/albertslund/2620/lejlighed/2750000492-275/
https://danbolig.dk/bolig/rudersdal/2970/villa/1980000832-198/
https://danbolig.dk/bolig/roskilde/413

## Load the data from the links
Get zip code by regex, to separate roadname, zipcode and city in data


In [18]:
with open('all_links.txt', 'r', encoding='utf-8') as file:
    url_list = file.read().splitlines()


re_zip = r'https://danbolig\.dk/bolig/([^/]+)/(\d+)/'

zip_list = []
table_large = []
table_left = []
table_address = []
top_info_table = []
house_description = []
house_zip = []
house_data = []

def save_dataframe(frame, file_name):
    csv_filename = '{}.csv'.format(file_name)

    # Export the DataFrame to CSV
    frame.to_csv(csv_filename, index=False, sep=';')
    
def get_page_content(url_list):
    for url in url_list:
        match = re.search(re_zip, url)
        if match:
            city = match.group(1)
            zip_code = match.group(2)
            zip_list.append(zip_code)
        try:
            print(url)
            # Send a request to the URL and get the content of the webpage
            response = requests.get(url, headers={'name': 'August Jensen', 'email': 'blz957@alumni.ku.dk', 'institution': 'University of Copenhagen'})
            soup = BeautifulSoup(response.content, 'lxml')

            # Access the large main table and add it to the list of large_tables
            info_table_large = soup.find_all('table')[0].find('tbody')
            table_large.append(info_table_large)

            # Access the small table to the left and add it to the list of left_tables
            info_table_small_left = soup.find_all('table')[1].find('tbody')
            table_left.append(info_table_small_left)

            # Access the small info table at the top (address) and add it to the list of top_addresses
            upper_info_table = soup.find_all('div', class_='o-propertyHero__info o-propertyHero__info--desktop')[0]

            # Find the addresses
            addresses = upper_info_table.find('h1')
            table_address.append(addresses)

            # Find house price and other stats
            info_table = upper_info_table.find_all('ul', class_='o-propertyHero__facts')[0]
            top_info_table.append(info_table)

            #Get house description
            description_tags = soup.find_all('div', class_='o-content o-propertyDescription__description a-paragraph')
            print("Number of description tags found:", len(description_tags)) 
            
            for description in description_tags:
                house_description.append(description.get_text(strip=True))
            if len(description_tags) == 0:
                house_description.append('No description available')
            time.sleep(1)
        except IndexError:
            print("IndexError: Skipping this link.")
            continue


def get_correct_address(row):
    address_parts = row['address'].split()
    try:
        # Find the index of the zip code in the address parts
        zip_index = address_parts.index(row['zip code'])
        
        # Join the parts to the left of the zip code as the road
        road = ' '.join(address_parts[:zip_index])
        
        # Join the parts to the right of the zip code as the city
        city = ' '.join(address_parts[zip_index + 1:])
    except ValueError:
        city = address_parts[-1]
        zip_code = address_parts[-2]
        
        # If the city is followed by a single letter, adjust parts accordingly
        if len(city) == 1 and len(address_parts) >= 3:
            city = address_parts[-2] + ' ' + city
            zip_code = address_parts[-3]
            road = ' '.join(address_parts[:-3])
        else:
            road = ' '.join(address_parts[:-2])
    return pd.Series({'City': city, 'zip code': row['zip code'], 'Road': road})


def pass_content_to_dataframe():

    for house in range(len(table_large)):
        main_table_dict = {}

        main_table = table_large[house].find_all('td')
        for i in range(0, len(main_table), 2):
            name = main_table[i].text
            value = main_table[i + 1].text
            value = re.sub(r'\s*(m²|kr\.)', '', value)
            main_table_dict[name] = value

        top_table = top_info_table[house].find_all('span')
        for i in range(0, len(top_table), 2):
            name = top_table[i].text
            value = top_table[i + 1].text
            value = re.sub(r'\s*(m²|kr\.)', '', value)
            if name != 'Energimærke':
                main_table_dict[name] = value.strip()

        name = "address"
        value = ' '.join(table_address[house].text.split())
        main_table_dict[name] = value

        # Add description and zip_code fields from the provided lists
        main_table_dict['description'] = house_description[house]
        main_table_dict['zip code'] = zip_list[house]

        house_data.append(main_table_dict)

        

    df = pd.DataFrame(house_data)
       
    df[['City', 'zip code', 'Road']] = df.apply(get_correct_address, axis=1)

    save_dataframe(df, 'house_info_scraped')
    
    return df


## Get information on houses
Then edit adress format, and save to csv file


# Geotag and Distance Parameter


In [62]:
from geopy.geocoders import Nominatim
import time


df1 = pd.read_csv('house_info_scraped.csv',sep=';')
df = df1.copy()
df = df.loc[:100,:]
df.head()
# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")
from geopy.geocoders import GoogleV3
df['Road1'] = df['Road'].str.split(',', 1).str[0]
df['City'] = df['City'].astype(str)
df['full_address'] = df['Road1'] + ' ' + df['City']
df['full_address'] = df['full_address'].astype(str)
df['latitude'] = 0
df['longitude'] = 0

count = 0 
for index, row in df.iterrows():
    address = row['full_address']
    try:
        location = geolocator.geocode(address)
        if location is not None:
            df.at[index, 'latitude'] = location.latitude
            df.at[index, 'longitude'] = location.longitude
            print( index, location.longitude, location.latitude)
        else:
            df.at[index, 'latitude'] = np.nan
            df.at[index, 'longitude'] = np.nan
            print(address)
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
    #time.sleep(1)

save_dataframe(df, 'house_info_scraped_with_geocodes')
# Keep working in the next bucket of code

C:\Users\Marti\AppData\Local\Temp\ipykernel_8776\2347382426.py:5: DtypeWarning: Columns (181,182,183) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('house_info_scraped.csv',sep=';')
C:\Users\Marti\AppData\Local\Temp\ipykernel_8776\2347382426.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df['Road1'] = df['Road'].str.split(',', 1).str[0]


0 9.541604 55.766191
1 9.561087 55.682778
2 9.546171 55.699203
3 9.535063 55.712654
4 9.545488 55.700245
5 9.581329 55.666628
6 9.516251 55.694327
7 9.554717 55.698051
8 9.54568 55.681276
9 9.677746 55.663318
10 9.571871 55.751932
11 9.547099 55.76429
12 9.500105 55.72426
13 9.54633 55.730473
14 9.55266 55.696788
15 9.537686 55.674088
16 8.489325 55.947347
17 9.374131 55.751031
18 9.519019 55.701509
19 9.523059 55.68913
20 9.556315 55.679684
21 9.555508 55.716415
22 9.405667 55.662697
23 9.550096 55.678127
24 9.504517 55.725135
25 9.543002 55.69772
26 9.542263 55.761408
27 9.532375 55.719987
Sindballevej 16+18 Vejle
29 9.576595 55.796018
30 9.562469 55.687595
31 9.518567 55.786021
32 9.452838 55.522499
33 9.546194 55.499088
34 9.520694 55.498997
35 9.509853 55.501758
36 9.455077 55.487852
37 9.465633 55.498287
38 9.519946 55.484331
39 9.473118 55.453196
40 9.438213 55.492566
41 9.444381 55.478865
42 9.450158 55.476225
43 9.523677 55.497888
44 9.510212 55.50686
45 9.470854 55.499383
46 

# Get nearest city

In [69]:
from geopy.distance import geodesic
import pandas as pd
import numpy as np

df= pd.read_csv('house_info_scraped_with_geocodes.csv',sep=';')

# Sample cities and their coordinates
cities = {
    'Copenhagen': (55.6761, 12.5683),
    'Aarhus': (56.1629, 10.2039),
    'Aalborg': (57.0480, 9.9197),
    'Odense': (55.3962, 10.3909)
}

def calculate_distance_to_city(row):
    geopoint = (row['latitude'], row['longitude'])
    min_distance = float('inf')  # Initialize with a large value
    closest_city = None

    for city, coordinates in cities.items():
        try:
            distance = geodesic(geopoint, coordinates).kilometers
        except ValueError:
            distance = np.nan
        if distance < min_distance:
            min_distance = distance
            closest_city = city
    
    return pd.Series({'distance to city': min_distance, 'closest city': closest_city})


#df['latitude'] = df['latitude'].astype(float)
#df['longitude'] = df['longitude'].astype(float)

# Keep working with the dataframe 'df' from the former code bucket:
# Apply the function to calculate distances and closest city, and merge the results
df[['distance to city', 'closest city']] = df.apply(calculate_distance_to_city, axis=1)


# Columns to format
columns_to_format = ['longitude','distance to city', 'latitude']

# Format the specified columns with two decimal places and use a comma as decimal separator

df[columns_to_format] = df[columns_to_format].applymap(lambda x: f'{x:.5f}'.replace('.', ','))

#df.head(20)
# Save the updated DataFrame

    # Export the DataFrame to CSV
df.to_csv('house_info_scraped_with_geocodes_and_city.csv', index=False,sep=';',decimal=',')



# DinGeo Data / tinglysning
